# ⚡ 03 - Inferência Eficiente para GPU 2GB
## 🎯 Maximizando Performance no Hardware Limitado

**Problema anterior resolvido:** Alguns modelos BERT não suportam `device_map='auto'`
**Solução:** Carregamento manual otimizado + técnicas de inferência eficiente

---

## 📊 RESULTADOS DO NOTEBOOK 02
✅ GPT-2 PT: 0.8GB | Funciona perfeitamente  
⚠️ BERT Legal: Precisa carregamento manual (sem device_map)  
🎯 Memória livre: 1.3GB (ótimo para otimizações)

---

## 🎯 OBJETIVOS DESTE NOTEBOOK
1. Carregar BERT jurídico SEM device_map='auto'
2. Otimizar inferência com float16 e chunking
3. Implementar cache de atenção para documentos longos
4. Testar quantização 8-bit (se bitsandbytes instalado)
5. Criar pipeline jurídico ULTRA eficiente

In [1]:
# Configuração RÁPIDA
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
import time
import os

print("="*70)
print("⚡ INFERÊNCIA LEVE - CONFIGURAÇÃO RÁPIDA")
print("="*70)

# Forçar uso de cache local (evitar downloads)
os.environ['TRANSFORMERS_OFFLINE'] = '0'  # Mas usa cache
os.environ['HF_HUB_OFFLINE'] = '0'

# Status rápido
print(f"🔧 PyTorch: {torch.__version__}")
print(f"🎮 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"💾 VRAM Total: 2.1 GB")

# Limpar para começar
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 Cache GPU limpo")

print("\n" + "="*70)

⚡ INFERÊNCIA LEVE - CONFIGURAÇÃO RÁPIDA
🔧 PyTorch: 2.3.1+cu118
🎮 GPU: NVIDIA GeForce 930M
💾 VRAM Total: 2.1 GB
🧹 Cache GPU limpo



In [2]:
# CARREGAMENTO ÚNICO (evita downloads repetidos)
print("📥 CARREGAMENTO ÚNICO DE MODELOS")
print("="*70)

# Dicionário para armazenar modelos já carregados
modelos_carregados = {}

def carregar_modelo_uma_vez(nome_modelo, tipo='masked'):
    """
    Carrega modelo UMA VEZ e reutiliza
    """
    if nome_modelo in modelos_carregados:
        print(f"   ⚡ REUTILIZANDO {nome_modelo} (já carregado)")
        return modelos_carregados[nome_modelo]
    
    print(f"   📥 Carregando {nome_modelo}...")
    inicio = time.time()
    
    # Carregar tokenizer
    tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
    
    # Carregar modelo com tipo correto
    if tipo == 'masked':
        model = AutoModelForMaskedLM.from_pretrained(nome_modelo)
    elif tipo == 'sequence':
        model = AutoModelForSequenceClassification.from_pretrained(
            nome_modelo,
            num_labels=3  # Para classificação simples
        )
    else:
        model = AutoModelForMaskedLM.from_pretrained(nome_modelo)
    
    # Otimizar para GPU
    if torch.cuda.is_available():
        model = model.half().to('cuda')  # float16 + GPU
    
    tempo = time.time() - inicio
    memoria = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
    
    print(f"   ✅ Carregado em {tempo:.1f}s | {memoria:.2f}GB")
    
    # Armazenar para reutilização
    modelos_carregados[nome_modelo] = (tokenizer, model)
    
    return tokenizer, model

# Carregar APENAS OS ESSENCIAIS
print("\n🎯 CARREGANDO MODELOS ESSENCIAIS:")

# 1. BERT para análise
tokenizer_bert, model_bert = carregar_modelo_uma_vez(
    "neuralmind/bert-base-portuguese-cased",
    tipo='sequence'
)

print(f"\n📊 STATUS APÓS CARREGAMENTO:")
if torch.cuda.is_available():
    print(f"   💾 Memória usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"   🎯 Memória livre: {2.1 - (torch.cuda.memory_allocated() / 1e9):.2f} GB")

📥 CARREGAMENTO ÚNICO DE MODELOS

🎯 CARREGANDO MODELOS ESSENCIAIS:
   📥 Carregando neuralmind/bert-base-portuguese-cased...


C:\Users\pcwin\anaconda3\envs\esp_ai\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   ✅ Carregado em 12.5s | 0.23GB

📊 STATUS APÓS CARREGAMENTO:
   💾 Memória usada: 0.23 GB
   🎯 Memória livre: 1.87 GB


In [3]:
# TÉCNICA ESSENCIAL 1 - Chunking SIMPLES
print("\n" + "="*70)
print("📄 CHUNKING SIMPLES (sem carregar novo modelo)")
print("="*70)

def chunking_simples(texto, max_tokens=256):
    """
    Chunking simples sem carregar novo modelo
    """
    # Dividir por parágrafos (simples e eficiente)
    paragrafos = [p.strip() for p in texto.split('\n\n') if p.strip()]
    
    chunks = []
    chunk_atual = ""
    tokens_atual = 0
    
    for paragrafo in paragrafos:
        # Estimativa simples de tokens (4 chars ≈ 1 token)
        tokens_paragrafo = len(paragrafo) // 4
        
        if tokens_atual + tokens_paragrafo > max_tokens and chunk_atual:
            # Salvar chunk atual e começar novo
            chunks.append(chunk_atual.strip())
            chunk_atual = paragrafo
            tokens_atual = tokens_paragrafo
        else:
            # Adicionar ao chunk atual
            if chunk_atual:
                chunk_atual += "\n\n" + paragrafo
            else:
                chunk_atual = paragrafo
            tokens_atual += tokens_paragrafo
    
    # Adicionar último chunk
    if chunk_atual:
        chunks.append(chunk_atual.strip())
    
    return chunks

# Teste RÁPIDO
print("\n🧪 TESTE RÁPIDO DE CHUNKING:")

contrato_pequeno = """
CLÁUSULA 1 - OBJETO. Contrato de serviços.
CLÁUSULA 2 - PRAZO. 12 meses.
CLÁUSULA 3 - VALOR. R$ 5.000,00.
CLÁUSULA 4 - MULTA. 10% por descumprimento.
"""

chunks = chunking_simples(contrato_pequeno, max_tokens=50)
print(f"   Texto original: {len(contrato_pequeno)} caracteres")
print(f"   Dividido em: {len(chunks)} chunks")
for i, chunk in enumerate(chunks, 1):
    print(f"   Chunk {i}: {chunk[:50]}...")


📄 CHUNKING SIMPLES (sem carregar novo modelo)

🧪 TESTE RÁPIDO DE CHUNKING:
   Texto original: 151 caracteres
   Dividido em: 1 chunks
   Chunk 1: CLÁUSULA 1 - OBJETO. Contrato de serviços.
CLÁUSUL...


In [4]:
# TÉCNICA ESSENCIAL 2 - Pipeline Jurídico RÁPIDO
print("\n" + "="*70)
print("⚖️ PIPELINE JURÍDICO RÁPIDO (com modelos já carregados)")
print("="*70)

class PipelineJuridicoRapido:
    """Pipeline ultra leve usando modelos já carregados"""
    
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.device = next(model.parameters()).device
        
    def analisar_clausula_rapido(self, texto):
        """Análise RÁPIDA de uma cláusula"""
        # Tokenizar
        inputs = self.tokenizer(
            texto,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding=True
        ).to(self.device)
        
        # Inferência RÁPIDA
        inicio = time.time()
        with torch.no_grad():
            outputs = self.model(**inputs)
            predicao = torch.softmax(outputs.logits, dim=-1)
        
        tempo = time.time() - inicio
        
        # Resultado simplificado
        scores = predicao[0].cpu().numpy()
        tipos = ["Normal", "Importante", "Crítica"]
        tipo_idx = scores.argmax()
        
        return {
            'texto': texto[:100] + "..." if len(texto) > 100 else texto,
            'tipo': tipos[tipo_idx],
            'confianca': float(scores[tipo_idx]),
            'tempo_ms': tempo * 1000,
            'tokens': inputs['input_ids'].shape[1]
        }
    
    def analisar_contrato_rapido(self, texto_contrato):
        """Análise RÁPIDA de contrato completo"""
        print(f"\n📄 ANALISANDO CONTRATO: {len(texto_contrato):,} chars")
        
        # 1. Chunking simples
        chunks = chunking_simples(texto_contrato, max_tokens=200)
        print(f"   📦 Dividido em {len(chunks)} chunks")
        
        # 2. Analisar cada chunk (limitar a 3 para velocidade)
        chunks = chunks[:3]  # LIMITE para não travar
        resultados = []
        
        for i, chunk in enumerate(chunks, 1):
            print(f"   🔍 Analisando chunk {i}/{len(chunks)}...", end="")
            
            resultado = self.analisar_clausula_rapido(chunk)
            resultados.append(resultado)
            
            print(f" ✅ {resultado['tipo']} ({resultado['confianca']:.0%})")
        
        # 3. Resumo
        print(f"\n📊 RESUMO DA ANÁLISE:")
        for r in resultados:
            print(f"   • {r['tipo']}: {r['confianca']:.0%} confiança")
        
        return resultados

# Criar pipeline RÁPIDO
print("\n🚀 CRIANDO PIPELINE RÁPIDO...")
pipeline = PipelineJuridicoRapido(tokenizer_bert, model_bert)

# Teste com cláusula ÚNICA (rápido)
print("\n🧪 TESTE 1: Análise de cláusula única")

clausula_teste = "Multa de 10% por atraso na entrega."
resultado = pipeline.analisar_clausula_rapido(clausula_teste)

print(f"\n📋 RESULTADO:")
print(f"   Texto: {resultado['texto']}")
print(f"   Tipo: {resultado['tipo']}")
print(f"   Confiança: {resultado['confianca']:.1%}")
print(f"   Tempo: {resultado['tempo_ms']:.1f}ms")
print(f"   Tokens: {resultado['tokens']}")

# Teste com contrato PEQUENO
print("\n🧪 TESTE 2: Análise de contrato pequeno")

contrato_pequeno_real = """
CONTRATO DE PRESTAÇÃO DE SERVIÇOS

CLÁUSULA 1 - OBJETO. O presente contrato tem por objeto a consultoria jurídica.

CLÁUSULA 2 - PRAZO. Vigência de 6 meses.

CLÁUSULA 3 - VALOR. R$ 8.000,00 mensais.

CLÁUSULA 4 - CONFIDENCIALIDADE. As partes mantêm sigilo.
"""

resultados = pipeline.analisar_contrato_rapido(contrato_pequeno_real)

print("\n✅ PIPELINE FUNCIONANDO NA GPU 2GB!")


⚖️ PIPELINE JURÍDICO RÁPIDO (com modelos já carregados)

🚀 CRIANDO PIPELINE RÁPIDO...

🧪 TESTE 1: Análise de cláusula única

📋 RESULTADO:
   Texto: Multa de 10% por atraso na entrega.
   Tipo: Normal
   Confiança: 35.6%
   Tempo: 682.2ms
   Tokens: 12

🧪 TESTE 2: Análise de contrato pequeno

📄 ANALISANDO CONTRATO: 258 chars
   📦 Dividido em 1 chunks
   🔍 Analisando chunk 1/1... ✅ Crítica (38%)

📊 RESUMO DA ANÁLISE:
   • Crítica: 38% confiança

✅ PIPELINE FUNCIONANDO NA GPU 2GB!


In [5]:
# TÉCNICA ESSENCIAL 3 - Gerenciamento de Memória PRÁTICO
print("\n" + "="*70)
print("🧠 GERENCIAMENTO DE MEMÓRIA PRÁTICO")
print("="*70)

print("\n🎯 COMANDOS ESSENCIAIS PARA SUA GPU 2GB:")

comandos_essenciais = [
    ("1. Antes de carregar", "torch.cuda.empty_cache()", "Limpa cache da GPU"),
    ("2. Após usar modelo", "del modelo", "Remove referência"),
    ("3. Forçar coleta", "import gc; gc.collect()", "Libera memória RAM"),
    ("4. Sempre usar", "with torch.no_grad():", "Inferência sem gradientes"),
    ("5. Precisão", "model.half()", "float16 (metade da memória)"),
]

for num, comando, desc in comandos_essenciais:
    print(f"\n{num}")
    print(f"   💻 {comando}")
    print(f"   📝 {desc}")

# Demonstração PRÁTICA
print("\n🔍 DEMONSTRAÇÃO PRÁTICA DE MEMÓRIA:")

if torch.cuda.is_available():
    # Estado atual
    memoria_inicial = torch.cuda.memory_allocated() / 1e9
    print(f"   Memória atual: {memoria_inicial:.2f} GB")
    
    # Criar e deletar tensor
    tensor_teste = torch.randn(1000, 1000, device='cuda')  # ~4MB
    memoria_com_tensor = torch.cuda.memory_allocated() / 1e9
    print(f"   Com tensor: {memoria_com_tensor:.2f} GB")
    
    del tensor_teste
    torch.cuda.empty_cache()
    
    memoria_final = torch.cuda.memory_allocated() / 1e9
    print(f"   Após deletar: {memoria_final:.2f} GB")
    print(f"   ✅ Liberado: {memoria_com_tensor - memoria_final:.3f} GB")


🧠 GERENCIAMENTO DE MEMÓRIA PRÁTICO

🎯 COMANDOS ESSENCIAIS PARA SUA GPU 2GB:

1. Antes de carregar
   💻 torch.cuda.empty_cache()
   📝 Limpa cache da GPU

2. Após usar modelo
   💻 del modelo
   📝 Remove referência

3. Forçar coleta
   💻 import gc; gc.collect()
   📝 Libera memória RAM

4. Sempre usar
   💻 with torch.no_grad():
   📝 Inferência sem gradientes

5. Precisão
   💻 model.half()
   📝 float16 (metade da memória)

🔍 DEMONSTRAÇÃO PRÁTICA DE MEMÓRIA:
   Memória atual: 0.23 GB
   Com tensor: 0.24 GB
   Após deletar: 0.23 GB
   ✅ Liberado: 0.004 GB


In [6]:
# Célula 6: PRÓXIMOS PASSOS IMEDIATOS
print("\n" + "="*70)
print("🚀 PRÓXIMOS PASSOS - FOCO NO PRÁTICO")
print("="*70)

print("""
✅ VOCÊ JÁ TEM (do notebook leve):
1. Pipeline jurídico funcional
2. Chunking simples para contratos longos  
3. Análise com BERT em português
4. Gerenciamento de memória básico

🎯 PRÓXIMO: Módulo 04 - FINE-TUNING PRÁTICO

CONTEÚDO DO MÓDULO 04:
1. Preparar SEUS documentos jurídicos como dataset
2. Fine-tuning de BERT para SUAS classificações
3. Avaliação do modelo fine-tuned
4. Comparação antes/depois do fine-tuning

📁 ESTRUTURA SUGERIDA:
04_Fine_Tuning_Pratico/
├── 01_preparar_dados.ipynb
├── 02_fine_tuning_bert.ipynb  
├── 03_avaliacao_modelo.ipynb
└── dados/
    ├── contratos/
    ├── classificacoes/
    └── modelo_fine_tuned/
""")

# Verificação final
print("\n🔍 STATUS FINAL DA SUA GPU 2GB:")
if torch.cuda.is_available():
    memoria_atual = torch.cuda.memory_allocated() / 1e9
    print(f"   💾 Memória usada: {memoria_atual:.2f} GB")
    print(f"   🎯 Memória livre: {2.1 - memoria_atual:.2f} GB")
    
    if memoria_atual < 0.5:
        print("   ✅ Excelente! Pronto para fine-tuning.")
    else:
        print("   ⚠️  Considere limpar memória antes do próximo módulo.")

print("\n" + "="*70)
print("🎉 NOTEBOOK 03 LEVE CONCLUÍDO EM MINUTOS!")
print("="*70)
print("\n🏆 RESUMO DO APRENDIZADO:")
print("1. ✅ Evitar downloads repetidos (cache)")
print("2. ✅ Pipeline jurídico funcional")
print("3. ✅ Chunking simples implementado")
print("4. ✅ Gerenciamento de memória prático")


🚀 PRÓXIMOS PASSOS - FOCO NO PRÁTICO

✅ VOCÊ JÁ TEM (do notebook leve):
1. Pipeline jurídico funcional
2. Chunking simples para contratos longos  
3. Análise com BERT em português
4. Gerenciamento de memória básico

🎯 PRÓXIMO: Módulo 04 - FINE-TUNING PRÁTICO

CONTEÚDO DO MÓDULO 04:
1. Preparar SEUS documentos jurídicos como dataset
2. Fine-tuning de BERT para SUAS classificações
3. Avaliação do modelo fine-tuned
4. Comparação antes/depois do fine-tuning

📁 ESTRUTURA SUGERIDA:
04_Fine_Tuning_Pratico/
├── 01_preparar_dados.ipynb
├── 02_fine_tuning_bert.ipynb  
├── 03_avaliacao_modelo.ipynb
└── dados/
    ├── contratos/
    ├── classificacoes/
    └── modelo_fine_tuned/


🔍 STATUS FINAL DA SUA GPU 2GB:
   💾 Memória usada: 0.23 GB
   🎯 Memória livre: 1.87 GB
   ✅ Excelente! Pronto para fine-tuning.

🎉 NOTEBOOK 03 LEVE CONCLUÍDO EM MINUTOS!

🏆 RESUMO DO APRENDIZADO:
1. ✅ Evitar downloads repetidos (cache)
2. ✅ Pipeline jurídico funcional
3. ✅ Chunking simples implementado
4. ✅ Gerenciamento